In [1]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType

In [12]:
KEY = os.environ['COGNITIVE_SERVICE_KEY']
ENDPOINT = os.environ['ENDPOINT'] 

In [29]:
# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [78]:
# Used in the Person Group Operations,  Snapshot Operations, and Delete Person Group examples.
# You can call list_person_groups to print a list of preexisting PersonGroups.
# SOURCE_PERSON_GROUP_ID should be all lowercase and alphanumeric. For example, 'mygroupname' (dashes are OK).
PERSON_GROUP_ID = 'imm_001'

# Used for the Snapshot and Delete Person Group examples.
TARGET_PERSON_GROUP_ID = str(uuid.uuid4()) # assign a random ID (or name it anything)

In [76]:
train_path = '../imm/train_min/'
ids = set([x.split('-')[0] for x in os.listdir(train_path)])
ids = list(ids)
ids.sort()

In [79]:
'''
Create the PersonGroup
'''
# Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
print('Person group:', PERSON_GROUP_ID)
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

people = []
index_dict = []
for index in ids:
    person = face_client.person_group_person.create(PERSON_GROUP_ID, index)
    people.append(person)
    index_dict.extend([{'index': index, 'person_id': person.person_id}])


Person group: imm_001
01
a1217bc6-0644-4d99-8eb3-9ee12b4b70a2
02
ca82a100-4937-418e-9e40-4e69e068bd93
03
2eea2864-b79e-4981-95b7-944aa483502b
04
aecce438-4b46-4a4b-a656-a216e7d92a6d
05
50d1a4f7-1cf7-48be-a2ec-0939c420bdb2
06
c31df498-bfc9-4558-995d-ee8fae900034
07
c0bb0c30-62ba-4fbd-bbba-a83f0a30c5c4
08
79c31334-b282-457c-815b-86a22e3b4434
09
f16c1b4d-0811-457e-9209-ff46a87e9031
10
0f34f2c2-adb4-4a97-a20a-971156b2a57f
11
6565c380-fdde-46fc-b28f-db2bdafaf981
12
8dded720-8b72-475e-9d06-8fb60a61795e
13
290655ef-516c-4b41-8e17-98e29f9621b7
14
114e6fc2-1e94-4506-8ca7-1934f7a5d7e4
15
3edd7dc0-8f85-4bb0-8df9-8b2393d84e60
16
85b69a49-64ec-4041-aaad-8735b08519e2
17
c918f739-8643-495e-8f64-336e84238bc6
18
4a292511-cda4-4be7-9118-34c156425913
19
be7b3505-10ae-4227-98bc-ae7352517d37
20
78ecc3e2-ab7a-4cfd-88c5-98e2b158465d
21
6062ffd7-4252-4e42-b5e0-90892b6b79c3
22
14f7eef2-4b30-4f95-a19b-123bf7a3ac78
23
8ca790b2-ed45-449a-850a-5d1e60c4ee01
24
116444b5-5609-48b5-afce-ea81734e2aed
25
5699571a-0257-4

In [80]:
'''
Detect faces and register to correct person
'''
# Find all jpeg images of person in working directory
people_images = []
for index in ids:
    person_images = [train_path + file for file in os.listdir(train_path) if file.startswith(index)]
    people_images.append(person_images)

for person, person_images in zip(people, people_images):
    # Add to a woman person
    for image in person_images:
        w = open(image, 'r+b')
        face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, person.person_id, w)


In [81]:
'''
Train PersonGroup
'''
print()
print('Training the person group...')
# Train the person group
face_client.person_group.train(PERSON_GROUP_ID)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        sys.exit('Training the person group has failed.')
    time.sleep(5)


Training the person group...
Training status: running.

Training status: succeeded.



In [87]:
os.listdir('../imm/test/')

['17-6m.jpg',
 '35-6f.jpg',
 '05-6m.jpg',
 '22-6f.jpg',
 '26-6m.jpg',
 '34-6m.jpg',
 '31-6m.jpg',
 '38-6m.jpg',
 '02-6m.jpg',
 '06-6m.jpg',
 '07-6m.jpg',
 '11-6m.jpg',
 '32-6m.jpg',
 '30-6f.jpg',
 '39-6m.jpg',
 '20-6m.jpg',
 '16-6m.jpg',
 '18-6m.jpg',
 '36-6m.jpg',
 '37-6m.jpg',
 '12-6f.jpg',
 '21-6m.jpg',
 '14-6f.jpg',
 '23-6m.jpg',
 '19-6m.jpg',
 '28-6m.jpg',
 '15-6f.jpg',
 '33-6m.jpg',
 '24-6m.jpg',
 '25-6m.jpg',
 '29-6m.jpg',
 '03-6m.jpg',
 '04-6m.jpg',
 '40-6m.jpg',
 '13-6m.jpg',
 '08-6f.jpg',
 '01-6m.jpg',
 '27-6m.jpg',
 '10-6m.jpg',
 '09-6m.jpg']

In [88]:
'''
Identify a face against a defined PersonGroup
'''
TEST_FOLDER = '../imm/test/'
for test_image in os.listdir(TEST_FOLDER):
    # Get test image
    test_image_array = glob.glob(os.path.join(IMAGES_FOLDER, test_image))
    image = open(test_image_array[0], 'r+b')

    # Detect faces
    face_ids = []
    faces = face_client.face.detect_with_stream(image)
    for face in faces:
        face_ids.append(face.face_id)

    # Identify faces
    results = face_client.face.identify(face_ids, PERSON_GROUP_ID)
    print('Identifying faces in {}'.format(os.path.basename(image.name)))
    if not results:
        print('No person identified in the person group for faces from {}.'.format(os.path.basename(image.name)))
    for person in results:
        index_person = [d['index'] for d in index_dict if d['person_id']== person.candidates[0].person_id]
        if index_person[0] == os.path.basename(image.name).split('-')[0]:
            print('Succesfull identifed with confidence: ', person.candidates[0].confidence)

Identifying faces in 17-6m.jpg
Succesfull identifed with confidence:  0.70441
Identifying faces in 35-6f.jpg
Succesfull identifed with confidence:  0.71688
Identifying faces in 05-6m.jpg
Succesfull identifed with confidence:  0.76879
Identifying faces in 22-6f.jpg
Succesfull identifed with confidence:  0.69649
Identifying faces in 26-6m.jpg
Succesfull identifed with confidence:  0.87816
Identifying faces in 34-6m.jpg
Succesfull identifed with confidence:  0.78223
Identifying faces in 31-6m.jpg
Succesfull identifed with confidence:  0.74775
Identifying faces in 38-6m.jpg
Succesfull identifed with confidence:  0.71601
Identifying faces in 02-6m.jpg
Succesfull identifed with confidence:  0.78883
Identifying faces in 06-6m.jpg
Succesfull identifed with confidence:  0.86857
Identifying faces in 07-6m.jpg
Succesfull identifed with confidence:  0.7365
Identifying faces in 11-6m.jpg
Succesfull identifed with confidence:  0.79475
Identifying faces in 32-6m.jpg
Succesfull identifed with confiden